In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "pwsh -c "../../../../scripts/invoke-dib.ps1 test.dib"", [||], None,
        None, None)
00:00:01 verbose #2 > core.Invoke-Dib / Get-Location: C:\home\git\polyglot\apps\spiral\temp\test / path: test.dib / _args: null
00:00:03 verbose #3 >
00:00:03 verbose #4 > ── markdown ────────────────────────────────────────────────────────────────────
00:00:03 verbose #5 > ╭──────────────────────────────────────────────────────────────────────────────╮
00:00:03 verbose #6 > │ # test (Polyglot)                                                            │
00:00:03 verbose #7 > ╰──────────────────────────────────────────────────────────────────────────────╯
00:00:03 verbose #8 >
00:00:03 verbose #9 > ── spiral ──────────────────────────────────────────────────────────────────────
00:00:03 verbose #10 > //// test
00:00:03 verbose #

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00   debug #1 writeDibCode / output: Spi / path: test.dib
00:00:00   debug #2 parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #2 run_with_timeout_async / timeout: 500
00:00:01   debug #1 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01   debug #2 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01 verbose #3 sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnomin...0022\u003C/test\u003E\u0022\n","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:01 verbose #4 sendJson / port: 13805 / json: {"BuildFile":{"backend":"Fsharp","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:02   debug #5 buildFile / takeWhileInclusive / path: test.spi / fsxContent: let rec closure0 () () : int32 =
    let v0 : string = "test"
    System.Console.WriteLine v0
    0
let v0 : (unit -> int32) = closure0()
()


In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.17.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @zpodlovics
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> cmd /C dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fable-temp.csproj (in 1.47 sec).
.> cmd /C dotnet restore C:/home/git/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fsproj (in 293 ms).
Project and references (1 source files) parsed in 9438ms

Started Fable compilation...

Fable compilation finished in 2311ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

    Blocking waiting for file lock on package cache
    Blocking waiting for file lock on package cache
   Compiling getrandom v0.2.14
   Compiling once_cell v1.19.0
   Compiling winapi v0.3.9
   Compiling fastrand v2.1.0
   Compiling num-traits v0.2.18
   Compiling ppv-lite86 v0.2.17
   Compiling syn v2.0.60
   Compiling windows-sys v0.52.0
   Compiling rand_core v0.6.4
   Compiling tracing-core v0.1.32
   Compiling lazy_static v1.4.0
   Compiling rand_chacha v0.3.1
   Compiling libm v0.2.8
   Compiling quick-error v1.2.3
   Compiling tempfile v3.10.1
   Compiling overload v0.1.1
   Compiling thiserror v1.0.59
   Compiling fnv v1.0.7
   Compiling log v0.4.21
   Compiling bit-vec v0.6.3
   Compiling wait-timeout v0.2.0
   Compiling tracing-log v0.2.0
   Compiling rusty-fork v0.3.0
   Compiling bit-set v0.5.3
   Compiling rand v0.8.5
   Compiling sharded-slab v0.1.7
   Compiling nu-ansi-term v0.46.0
   Compiling rand_xorshift v0.3.0
   Compiling thread_local v1.1.8
   Compiling bitflags

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling once_cell v1.19.0
   Compiling fastrand v2.1.0
   Compiling winapi v0.3.9
   Compiling bit-vec v0.6.3
   Compiling overload v0.1.1
   Compiling windows-sys v0.52.0
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling tracing-core v0.1.32
   Compiling fnv v1.0.7
   Compiling log v0.4.21
   Compiling bit-set v0.5.3
   Compiling thread_local v1.1.8
   Compiling rand v0.8.5
   Compiling num-traits v0.2.18
   Compiling sharded-slab v0.1.7
   Compiling rand_xorshift v0.3.0
   Compiling tracing-log v0.2.0
   Compiling bitflags v2.5.0
   Compiling tempfile v3.10.1
   Compiling memchr v2.7.2
   Compiling unarray v0.1.4
   Compiling minimal-lexical v0.2.1
   Compiling tracing v0.1.40
   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling thiserror v1.0.59
   Compiling rusty-fork v0.3.0
   Compiling nu-ansi-term v0.46.0
   Compiling nom v7.1.3
   Compiling proptest v1.4.0
   Compiling tracing

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
